In [1]:
import mysql.connector
from mysql.connector import errorcode

In [2]:
class MySQLConnection:
    
    def __init__(self, username, database, host, password='', port=3306):
        self.connectdict = {"host": host, "port": int(port), "user": username, "password": password, "database": database}
        self.dbconnection = self.connect()
        
    def connect(self):
        dbconnection = None
        try:
            dbconnection = mysql.connector.connect(**self.connectdict) 
            if dbconnection.is_connected():
                return dbconnection
            else:
                return "Cannot establish connection"
            
        except mysql.connector.Error as err:
            print(err.errno)
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Wrong username or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
            
        finally:
            if dbconnection is not None and dbconnection.is_connected():
                dbconnection.close()
                
                
    def query(self, statement, exception_when_empty=False):
        try:
            dbconnection = self.connect()        #dbconnection.is_connected() => False
            dbconnection.connect()
            cursor = dbconnection.cursor()
            cursor.execute(statement)

            result = []
            column_names = cursor.column_names
            for row in cursor:
                result.append(dict(zip(column_names, row)))

            if not result and exception_when_empty:
                return "No data returned"
            
        except Exception as e:
            print(e)
            
        finally:
            cursor.close()
            dbconnection.close()  
        return result
    
    def update(self, update_statement):
        try:
            dbconnection = self.connect()
            dbconnection.connect()
            cursor = dbconnection.cursor()
            cursor.execute(update_statement)
            dbconnection.commit()
        except Exception as e:
            print(e)
        finally:
            cursor.close()
            dbconnection.close()
        return
            
    def insert(self, insert_statement):
        try:
            dbconnection = self.connect()
            dbconnection.connect()
            cursor = dbconnection.cursor()
            cursor.execute(insert_statement)
            dbconnection.commit()
        except Exception as e:
            dbconnection.rollback()
            print(e)
        finally:
            cursor.close()
            dbconnection.close()
        return
    
    def delete(self, delete_statement):
        try:
            dbconnection = self.connect()
            dbconnection.connect()
            cursor = dbconnection.cursor()
            cursor.execute(delete_statement)
            dbconnection.commit()
        except Exception as e:
            dbconnection.rollback()
            print(e)
        finally:
            cursor.close()
            dbconnection.close()
        return
               

In [11]:
sqlconnection = MySQLConnection('root', 'mydatabase', 'localhost')

In [20]:
sqlconnection.insert('INSERT INTO employees (id, name, age, occupation) VALUES (1, "John", 28, "Engineer"), \
                    (2, "Jane", 24, "Doctor"), (3, "Bob", 32, "Artist")')


In [22]:
sqlconnection.query('SELECT * FROM employees')

[{'id': 1, 'name': 'John', 'age': 28, 'occupation': 'Engineer'},
 {'id': 2, 'name': 'Jane', 'age': 24, 'occupation': 'Doctor'},
 {'id': 3, 'name': 'Bob', 'age': 32, 'occupation': 'Artist'}]

In [23]:
sqlconnection.update('UPDATE employees SET Age = 25 WHERE id=2')

In [24]:
sqlconnection.query('SELECT * FROM employees')

[{'id': 1, 'name': 'John', 'age': 28, 'occupation': 'Engineer'},
 {'id': 2, 'name': 'Jane', 'age': 25, 'occupation': 'Doctor'},
 {'id': 3, 'name': 'Bob', 'age': 32, 'occupation': 'Artist'}]

In [25]:
sqlconnection.delete('DELETE FROM employees WHERE id=3')

In [26]:
sqlconnection.query('SELECT * FROM employees')

[{'id': 1, 'name': 'John', 'age': 28, 'occupation': 'Engineer'},
 {'id': 2, 'name': 'Jane', 'age': 25, 'occupation': 'Doctor'}]

In [9]:
def select(tablename, select_dict, where_dict=None):
    query = "Select "
    for select_col, alias in select_dict.items():
        if list(select_dict.keys()).index(select_col) < len(select_dict)-1 :
            s = f'{select_col} as {alias}, '
        else:
            s = f'{select_col} as {alias} '
        query += s
    query += f'From {tablename} '
    
    if where_dict:
        query += 'Where '
        for where_col, where_value in where_dict.items():
            if isinstance(where_value, tuple):
                query += f'{where_col} in {where_value} and '
            elif isinstance(where_value, str) or isinstance(where_value, int):
                query += f'{where_col} = {where_value} and '
            elif isinstance(where_value, dict):
                for operator, operand in where_value.items():
                    query += f'{where_col} {operator} {operand} and '
    query = query[:-4]
        
    return query

In [30]:
query = select('employees',  # Table Name 
               {"id": "ID", "name": "Name", "occupation": "Job"},  # Select dict  
               {"id": {"<=": 5}, "age": 25, "name": ("Jane", "Bob")}) # Where dict

query

"Select id as ID, name as Name, occupation as Job From employees Where id <= 5 and age = 25 and name in ('Jane', 'Bob') "

In [31]:
sqlconnection.query(query)

[{'ID': 2, 'Name': 'Jane', 'Job': 'Doctor'}]